In [62]:
import pandas as pd
import random
import numpy as np
import os

SEED = 123

In [28]:
outcome_df = pd.read_csv('/home/ngsci/datasets/brca-psj-path/v2/outcomes.csv')
map_df = pd.read_csv('/home/ngsci/datasets/brca-psj-path/v2/slide-biopsy-map.csv')
#Unique labels ['IA', 'IIB', 'IIA', '0', nan, 'IIIC', 'IV', 'IIIA', 'IIIB', 'IB']
label_mapping = {'IA':1, 'IB':1, 'IIA':2, 'IIB':2, 'IIIA':3, 'IIIB':3, 'IIIC':3, 'IV':4, '0':0}
outcome_df['label'] = outcome_df['stage'].map(label_mapping)
outcome_df = outcome_df[~pd.isna(outcome_df['label'])]

In [63]:
#stratified_df = outcome_df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 50)))
train_N = 300
train_df = outcome_df.groupby('label', group_keys=False).apply(lambda x: x.sample(int(np.rint(train_N*len(x)/len(outcome_df))))).sample(frac=1, random_state=SEED).reset_index(drop=True)

test_N = 400
remained_df = outcome_df[~outcome_df['biopsy_id'].isin(stratified_df['biopsy_id'])]
test_df = remained_df.groupby('label', group_keys=False).apply(lambda x: x.sample(int(np.rint(test_N*len(x)/len(remained_df))))).sample(frac=1, random_state=SEED).reset_index(drop=True)

In [91]:
train_mapping = map_df[map_df['biopsy_id'].isin(train_df['biopsy_id'])]
test_mapping = map_df[map_df['biopsy_id'].isin(test_df['biopsy_id'])]

path_prefix = '/home/ngsci/datasets/brca-psj-path/basic-downsampling/v2/'
train_mapping['downsampled_path'] = path_prefix + train_mapping['slide_id'] + '.png'
test_mapping['downsampled_path'] = path_prefix + test_mapping['slide_id'] + '.png'

/tmp/ipykernel_229/3022976989.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_mapping['downsampled_path'] = path_prefix + train_mapping['slide_id'] + '.png'
/tmp/ipykernel_229/3022976989.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_mapping['downsampled_path'] = path_prefix + test_mapping['slide_id'] + '.png'


In [93]:
check_res = train_mapping['downsampled_path'].map(lambda x: os.path.isfile(x))
assert(check_res.sum() == len(check_res))

check_res = test_mapping['downsampled_path'].map(lambda x: os.path.isfile(x))
assert(check_res.sum() == len(check_res))

In [94]:
train_df.to_csv('./train_outcomes.csv', index=False)
test_df.to_csv('./test_outcomes.csv', index=False)
train_mapping.to_csv('./train_mapping.csv', index=False)
test_mapping.to_csv('./test_mapping.csv', index=False)

In [96]:
test_mapping

,slide_id,biopsy_id,slide_path,downsampled_path
18,76972642-d4a8-401d-a9b1-896837ceb6f4,e3ab4000-96d8-4f26-affe-acde4ff75313,/home/ngsci/datasets/brca-psj-path/ndpi/2018/7...,/home/ngsci/datasets/brca-psj-path/basic-downs...
27,64896555-b3fb-4836-8745-303022a51c0a,c30af30f-9c1f-40c5-9f5e-c72202614255,/home/ngsci/datasets/brca-psj-path/ndpi/2018/6...,/home/ngsci/datasets/brca-psj-path/basic-downs...
43,aaf1d6eb-ae74-4dde-a0da-939b4c75f661,1e3d015d-14de-423f-a9c6-b1a64b50ee55,/home/ngsci/datasets/brca-psj-path/ndpi/2018/a...,/home/ngsci/datasets/brca-psj-path/basic-downs...
47,1a5edb6b-c75e-4534-9043-411836af2715,2c87cff1-f8b1-4fa5-9770-f03596954b54,/home/ngsci/datasets/brca-psj-path/ndpi/2018/1...,/home/ngsci/datasets/brca-psj-path/basic-downs...
58,63dfffd2-e0fb-4913-b894-757cdd02be6a,aa213ae3-5151-4902-9016-4488f08a59f9,/home/ngsci/datasets/brca-psj-path/ndpi/2018/6...,/home/ngsci/datasets/brca-psj-path/basic-downs...
...,...,...,...,...
52102,b4c5e016-8bac-436a-9fb0-79922007fe4c,3383381f-d257-4ce7-ad32-8f65d15b4522,/home/ngsci/datasets/brca-psj-path/ndpi/2015/b...,/home/ngsci/datasets/brca-psj-path/basic-downs...
52103,5ca6b6e6-1eb7-4f9c-bfbc-a780754207e8,3383381f-d257-4ce7-ad32-8f65d15b4522,/home/ngsci/datasets/brca-psj-path/ndpi/2015/5...,/home/ngsci/datasets/brca-psj-path/basic-downs...
52251,44847f12-bc6e-46b9-bb52-4280a180a01b,fc88911f-c3cd-41b6-969b-ed8a4562913b,/home/ngsci/datasets/brca-psj-path/ndpi/2015/4...,/home/ngsci/datasets/brca-psj-path/basic-downs...
52252,32a650d4-69a2-48c7-8687-f4ee76a9ea1d,fc88911f-c3cd-41b6-969b-ed8a4562913b,/home/ngsci/datasets/brca-psj-path/ndpi/2015/3...,/home/ngsci/datasets/brca-psj-path/basic-downs...
